<a href="https://colab.research.google.com/github/JunSeokCheon/kaggle_H-M_data_analysis/blob/master/del_noimage_underwear_swimwear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지가 없는 데이터와 underwear/swimwear류 데이터를 삭제하는 데이터 전처리를 수행합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/2nd_project_elice/inital_data

/content/drive/MyDrive/2nd_project_elice/inital_data


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
import os

In [4]:
articles = pd.read_csv("articles.csv")
customers = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions_train.csv")

In [5]:
print(articles.shape)
print(transactions.shape)

(105542, 25)
(31788324, 5)


In [6]:
test_articles = articles.copy()
test_transactions = transactions.copy()

In [7]:
print(test_articles.shape)
print(test_transactions.shape)

(105542, 25)
(31788324, 5)


In [8]:
ls

articles.csv  customers.csv  transactions_train.csv


In [9]:
## root : dir과 files가 있는 path
# dirs : root 아래에 있는 폴더들
# files : root 아래에 있는 파일들

# for root, dirs, files in os.walk('./images/'):
#   print(root)
#   print(dirs)
#   print(files)

### os 모듈을 활용하여 이미지 파일에서 이미지 이름을 추출합니다.
- "이미지 이름.jpg" 형태인지 확인하고, "654847001 (1)" 같은 중복 데이터를 확인한 후 이미지이름만 리스트에 저장합니다.
- 이미지 이름에서 맨 앞에 0을 빼면 article_id와 같기 때문에 이미지 이름만 저장합니다.

In [10]:
images_names = []
for base, dirs, files in tqdm(os.walk('../images/')):
    for _files in files:
      if len(_files.split(".jpg"))==2:
        if len(_files.split(".jpg")[0].split()) == 1: # "654847001 (1)" -> 이런 파일명을 걸러주기 위해(중복 데이터 제거)
          images_names.append(_files.split(".jpg")[0].split())

0it [00:00, ?it/s]

In [11]:
len(images_names)

105100

- 이미지 이름과 article_id의 데이터프레임을 만듭니다.
- 원래 articles 데이터에 새로운 데이터 프레임을 merge하면 이미지 이름이 없는 데이터(NaN)가 이미지가 없는 데이터라고 판단할 수 있습니다.

In [12]:
image_name_df = pd.DataFrame(images_names, columns = ["image_name"])
image_name_df["article_id"] = image_name_df["image_name"].apply(lambda x: int(x[1:]))
image_name_df.head()

,image_name,article_id
0,0194242049,194242049
1,0198518010,198518010
2,0194037001,194037001
3,0192460006,192460006
4,0194270044,194270044


In [13]:
image_article_df = test_articles[["article_id"]].merge(image_name_df, on=["article_id"], how="left")
print(image_article_df.shape)
image_article_df.head()

(105542, 2)


,article_id,image_name
0,108775015,0108775015
1,108775044,0108775044
2,108775051,0108775051
3,110065001,0110065001
4,110065002,0110065002


In [14]:
article_no_image_df = image_article_df.loc[image_article_df.image_name.isna()]
print(article_no_image_df.shape)
article_no_image_df.head()

(442, 2)


,article_id,image_name
88,174057028,NaN
117,179208001,NaN
258,212042043,NaN
259,212042066,NaN
261,212629004,NaN


In [15]:
test_articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


- product_group_name이 underwear인 데이터와 swimwear 데이터의 article_id를 리스트로 만들어서 저장합니다.
- 이미지가 없는 데이터의 article_id를 리스트로 만들어서 저장합니다.

In [35]:
no_image_articleid = article_no_image_df['article_id'].tolist()
no_image_articleid[:5]

[174057028, 179208001, 212042043, 212042066, 212629004]

In [33]:
article_underwear_del = test_articles[test_articles["product_group_name"] == "Underwear"]['article_id'].tolist()
article_underwear_del[:5]

[110065001, 110065002, 110065011, 114428026, 114428030]

In [34]:
article_swimwear_del = test_articles[(test_articles["product_group_name"] == "Swimwear") | (test_articles["product_group_name"] == "Unknown") | (test_articles["product_group_name"] == "Underwear/nightwear")]['article_id'].tolist()
article_swimwear_del[:5]

[156224002, 176550016, 176550017, 176550018, 176550020]

- articles.csv에서 이미지가 없는 데이터와 underwear 데이터를 삭제합니다.

In [36]:
for id in no_image_articleid:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [40]:
for id in article_underwear_del:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [44]:
for id in article_swimwear_del:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [50]:
test_articles.groupby(['product_group_name']).count()['article_id']

product_group_name
Accessories              11005
Bags                        25
Cosmetic                    48
Fun                          2
Furniture                   13
Garment Full body        13276
Garment Lower body       19770
Garment Upper body       42680
Garment and Shoe care        9
Interior textile             3
Items                       17
Nightwear                 1898
Shoes                     5156
Socks & Tights            2431
Stationery                   5
Name: article_id, dtype: int64

In [51]:
# 4~6월 거래내역 데이터만 사용(전체 데이터 2천만개라서 처리에 곤란)
test_transactions["t_dat"] = pd.to_datetime(test_transactions['t_dat'])
test_transactions = test_transactions.loc[test_transactions.t_dat >= pd.to_datetime('2020-04-01')] #
test_transactions = test_transactions.loc[test_transactions.t_dat <= pd.to_datetime('2020-06-01')] #

In [69]:
test_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
23934157,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808001,0.067780,2
23934158,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808007,0.067780,2
23934161,2020-04-01,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,783504004,0.019051,2
23934162,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,599580061,0.012034,2
23934163,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,844294002,0.019576,2


- transactions.csv에서 이미지가 없는 데이터와 underwear 데이터를 삭제합니다.
- customers.csv는 article에 대한 정보가 없기 때문에 제외해도 됩니다.

In [52]:
for id in no_image_articleid:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [62]:
for id in article_underwear_del:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [120]:
for id in article_swimwear_del:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [133]:
# 이미지 없는 것과 속옷류 데이터 삭제(2020-04 - 2020-06 데이터에서)
# article : 105542 -> 99638
# transaction : 31788324 -> 2491536
print(test_articles.shape)
print(test_transactions.shape)

(96338, 25)
(2101776, 5)


In [134]:
test_articles = test_articles.reset_index(drop=True)

In [135]:
test_articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
4,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96333,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
96334,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
96335,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
96336,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [136]:
test_transactions = test_transactions.reset_index(drop=True)

In [137]:
test_transactions

,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808001,0.067780,2
1,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808007,0.067780,2
2,2020-04-01,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,783504004,0.019051,2
3,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,762096002,0.022576,2
4,2020-04-01,000ec422ba5459f0295c1e86872d61b746bb8f84345efd...,759479001,0.006085,2
...,...,...,...,...,...
2101771,2020-06-01,fff887d056fff68679d6e2491588ee4e83e57bf077ca7a...,871674002,0.042356,2
2101772,2020-06-01,fff887d056fff68679d6e2491588ee4e83e57bf077ca7a...,879989002,0.010153,2
2101773,2020-06-01,fffb04448b4082028fcf3f205b05f0ee846da598eeca1f...,873977001,0.050831,2
2101774,2020-06-01,fffb04448b4082028fcf3f205b05f0ee846da598eeca1f...,851317001,0.042356,2


In [144]:
ls

customers.csv


In [145]:
test_articles.to_csv('articles.csv')
test_transactions.to_csv('transactions.csv')